<a href="https://colab.research.google.com/github/7201krap/PYTORCH_project/blob/main/seed100_neurons_batch1_512_to_768.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
mnist_trainset = torchvision.datasets.MNIST(root='./', train=True, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

mnist_testset  = torchvision.datasets.MNIST(root='./', 
                                train=False, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

train_dataloader = torch.utils.data.DataLoader(mnist_trainset, 
                                               batch_size=1, 
                                               shuffle=True)

test_dataloader  = torch.utils.data.DataLoader(mnist_testset, 
                                               batch_size=50, 
                                               shuffle=False)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
# ************* modify this section for later use *************
# Define the model 
class Model(torch.nn.Module):
    def __init__(self, neuron_size):
        super(Model, self).__init__()
        # modify this section for later use 
        self.linear_1 = torch.nn.Linear(784, neuron_size)
        self.linear_2 = torch.nn.Linear(neuron_size, 10)
        self.sigmoid12  = torch.nn.Sigmoid()

        self.layer_activations = dict()

    def forward(self, x):

        # modify this section for later use 
        x = x.reshape(x.size(0), -1)
        x = self.linear_1(x)
        x = self.sigmoid12(x)
        pred = self.linear_2(x)
        return pred
# ************* modify this section for later use *************

In [5]:
def get_activation(model, layer_name):    
    def hook(module, input, output):
        model.layer_activations[layer_name] = output
    return hook

In [6]:
def sparsity_calculator(final_spareness, neuron_size):
    sparseness_list_avg = list()
    sparseness_list_std = list()
    for single_epoch_spareness in final_spareness:

        hidden_layer_activation_list = single_epoch_spareness
        hidden_layer_activation_list = torch.stack(hidden_layer_activation_list)
        layer_activations_list = torch.reshape(hidden_layer_activation_list, (10000, neuron_size))

        layer_activations_list = torch.abs(layer_activations_list)  # modified 
        num_neurons = layer_activations_list.shape[1]
        population_sparseness = (np.sqrt(num_neurons) - (torch.sum(layer_activations_list, dim=1) / torch.sqrt(torch.sum(layer_activations_list ** 2, dim=1)))) / (np.sqrt(num_neurons) - 1)

        mean_sparseness_per_epoch = torch.mean(population_sparseness)
        std_sparseness_per_epoch  = torch.std(population_sparseness)

        sparseness_list_avg.append(mean_sparseness_per_epoch)

    return sparseness_list_avg

In [7]:
def selectivity(hidden_layer_each_neuron, neuron_size):
    __selectivity__ = list()
    # I will now try to find the average of each class for each neuron.
    # check out the next cell 
    avg_activations = [dict() for x in range(neuron_size)]
    for i, neuron in enumerate(hidden_layer_each_neuron):
        for k, v in neuron.items():
            # v is the list of activations for hidden layer's neuron k 
            avg_activations[i][k] = sum(v) / float(len(v))

    # generate 'neuron_size' lists to get only values in avg_activations
    only_activation_vals = [list() for x in range(neuron_size)]

    # get only values from avg_activations
    for i, avg_activation in enumerate(avg_activations):
        for value in avg_activation.values():
            only_activation_vals[i].append(value)


    for activation_val in only_activation_vals:
        # find u_max 
        u_max = np.max(activation_val)

        # find u_minus_max 
        u_minus_max = (np.sum(activation_val) - u_max) / 9

        # find selectivity 
        selectivity = (u_max - u_minus_max) / (u_max + u_minus_max)

        # append selectivity value to selectivity
        __selectivity__.append(selectivity)

    avg_selectivity = np.average(__selectivity__)
    std_selectivity = np.std(__selectivity__)
                                 
    return avg_selectivity, std_selectivity

In [8]:
# ************* modify this section for later use *************
def avg_std_calculator(_hidden_layer_each_neuron_12, neuron_size):

    avg_selectivity12, std_selectivity12 = selectivity(_hidden_layer_each_neuron_12, neuron_size)

    final_selectivity_avg = (avg_selectivity12) / 1
    final_selecvitity_std = (std_selectivity12) / 1

    return final_selectivity_avg, final_selecvitity_std
# ************* modify this section for later use *************

In [9]:
def model_factory(optimizer_name, seed_num, neuron_size):
    '''
    optimizer_name : choose one of Adagrad, Adadelta, SGD, and Adam 

    '''
    my_model = Model(neuron_size)
    print("my_model:", my_model)
    my_model.to(device)

    # ************* modify this section for later use *************
    my_model.sigmoid12.register_forward_hook(get_activation(my_model, 's12'))
    # ************* modify this section for later use *************

    if optimizer_name == 'Adadelta':
        my_optimizer = torch.optim.Adadelta(my_model.parameters(), lr=1.0)

    elif optimizer_name == 'Adagrad':
        my_optimizer = torch.optim.Adagrad(my_model.parameters(), lr=0.1)

    elif optimizer_name == 'SGD':
        my_optimizer = torch.optim.SGD(my_model.parameters(), lr=0.1)

    elif optimizer_name == 'Adam':
        my_optimizer = torch.optim.Adam(my_model.parameters(), lr=0.001)

    else:
        print("ERROR")
    
    print("my_optimizer:", my_optimizer)
    test_acc, sparsity_avg, selectivity_list_avg, selectivity_list_std = sparsity_selectivity_trainer(optimizer=my_optimizer, model=my_model, neuron_size=neuron_size)

    # ************* modify this section for later use *************
    file_saver = open(f"seed{seed_num}_batch1_neuronsize{neuron_size}_{optimizer_name}.txt", "w")
    # ************* modify this section for later use *************
    file_saver.write(str(test_acc)+'\n'+str(sparsity_avg)+'\n'+str(selectivity_list_avg)+'\n'+str(selectivity_list_std)+'\n\n')
    file_saver.close()

    if seed_num == 1:
        if neuron_size == 64:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1_batch1_neuronsize64_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1_batch1_neuronsize64_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1_batch1_neuronsize64_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1_batch1_neuronsize64_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 128:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1_batch1_neuronsize128_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1_batch1_neuronsize128_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1_batch1_neuronsize128_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1_batch1_neuronsize128_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 256:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1_batch1_neuronsize256_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1_batch1_neuronsize256_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1_batch1_neuronsize256_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1_batch1_neuronsize256_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 384:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1_batch1_neuronsize384_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1_batch1_neuronsize384_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1_batch1_neuronsize384_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1_batch1_neuronsize384_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 512:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1_batch1_neuronsize512_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1_batch1_neuronsize512_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1_batch1_neuronsize512_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1_batch1_neuronsize512_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 640:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1_batch1_neuronsize640_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1_batch1_neuronsize640_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1_batch1_neuronsize640_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1_batch1_neuronsize640_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 768:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1_batch1_neuronsize768_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1_batch1_neuronsize768_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1_batch1_neuronsize768_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1_batch1_neuronsize768_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

    elif seed_num == 100:
        if neuron_size == 64:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed100_batch1_neuronsize64_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed100_batch1_neuronsize64_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed100_batch1_neuronsize64_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed100_batch1_neuronsize64_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 128:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed100_batch1_neuronsize128_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed100_batch1_neuronsize128_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed100_batch1_neuronsize128_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed100_batch1_neuronsize128_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 256:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed100_batch1_neuronsize256_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed100_batch1_neuronsize256_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed100_batch1_neuronsize256_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed100_batch1_neuronsize256_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 384:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed100_batch1_neuronsize384_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed100_batch1_neuronsize384_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed100_batch1_neuronsize384_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed100_batch1_neuronsize384_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 512:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed100_batch1_neuronsize512_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed100_batch1_neuronsize512_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed100_batch1_neuronsize512_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed100_batch1_neuronsize512_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 640:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed100_batch1_neuronsize640_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed100_batch1_neuronsize640_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed100_batch1_neuronsize640_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed100_batch1_neuronsize640_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 768:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed100_batch1_neuronsize768_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed100_batch1_neuronsize768_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed100_batch1_neuronsize768_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed100_batch1_neuronsize768_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

    # seed 1234
    elif seed_num == 1234:
        if neuron_size == 64:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1234_batch1_neuronsize64_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1234_batch1_neuronsize64_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1234_batch1_neuronsize64_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1234_batch1_neuronsize64_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 128:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1234_batch1_neuronsize128_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1234_batch1_neuronsize128_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1234_batch1_neuronsize128_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1234_batch1_neuronsize128_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 256:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1234_batch1_neuronsize256_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1234_batch1_neuronsize256_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1234_batch1_neuronsize256_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1234_batch1_neuronsize256_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 384:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1234_batch1_neuronsize384_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1234_batch1_neuronsize384_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1234_batch1_neuronsize384_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1234_batch1_neuronsize384_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 512:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1234_batch1_neuronsize512_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1234_batch1_neuronsize512_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1234_batch1_neuronsize512_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1234_batch1_neuronsize512_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 640:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1234_batch1_neuronsize640_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1234_batch1_neuronsize640_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1234_batch1_neuronsize640_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1234_batch1_neuronsize640_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 768:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1234_batch1_neuronsize768_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1234_batch1_neuronsize768_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1234_batch1_neuronsize768_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1234_batch1_neuronsize768_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

In [10]:
no_epochs = 30
def sparsity_selectivity_trainer(optimizer, model, neuron_size):

    criterion = torch.nn.CrossEntropyLoss()
    train_loss = list()
    test_loss  = list()
    test_acc   = list()

    best_test_loss = 1

    selectivity_avg_list = list()
    selectivity_std_list = list()

    # ************* modify this section for later use *************
    final_spareness_12 = list()
    # ************* modify this section for later use *************

    for epoch in range(no_epochs):

        # ************* modify this section for later use *************
        hidden_layer_each_neuron_12 = [{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]} for x in range(neuron_size)]
        hidden_layer_each_neuron_12 = np.array(hidden_layer_each_neuron_12)
        # ************* modify this section for later use *************

        total_train_loss = 0
        total_test_loss = 0

        # ************* modify this section for later use *************
        hidden_layer_activation_list_12 = list()
        # ************* modify this section for later use *************

        # training
        # set up training mode 
        model.train()

        for itr, (images, labels) in enumerate(train_dataloader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            pred = model(images)

            loss = criterion(pred, labels)
            total_train_loss += loss.item()

            loss.backward()
            optimizer.step()

        # testing 
        # change to evaluation mode 
        model.eval()
        total = 0
        for itr, (images, labels) in enumerate(test_dataloader):
            images, labels = images.to(device), labels.to(device)

            pred = model(images)

            loss = criterion(pred, labels)
            total_test_loss += loss.item()

            # we now need softmax because we are testing.
            pred = torch.nn.functional.softmax(pred, dim=1)
            for i, p in enumerate(pred):
                if labels[i] == torch.max(p.data, 0)[1]:
                    total = total + 1

            # ***************** sparsity calculation ***************** #
            hidden_layer_activation_list_12.append(model.layer_activations['s12'])

            # ************* modify this section for later use *************
            for activation, label in zip(model.layer_activations['s12'], labels):
                label = label.item()
                with torch.no_grad():
                    activation = activation.numpy()
                for i in range(neuron_size):    
                    hidden_layer_each_neuron_12[i][label].append(activation[i])
    
        selectivity_avg, selecvitity_std = avg_std_calculator(hidden_layer_each_neuron_12, neuron_size)
        # ************* modify this section for later use *************
        
        selectivity_avg_list.append(selectivity_avg)
        selectivity_std_list.append(selecvitity_std)

        # this conains activations for all epochs 
        final_spareness_12.append(hidden_layer_activation_list_12)
        # ***************** sparsity calculation ***************** #

        # caculate accuracy 
        accuracy = total / len(mnist_testset)

        # append accuracy here
        test_acc.append(accuracy)

        print('\nEpoch: {}/{}, Train Loss: {:.8f}, Test Loss: {:.8f}, Test Accuracy: {:.8f}'.format(epoch + 1, no_epochs, total_train_loss, total_test_loss, accuracy))

    # ***************** sparsity calculation ***************** #
    sparsity_list12_avg = sparsity_calculator(final_spareness_12, neuron_size)

    average_sparsity = list()

    for i in range(no_epochs):
        average_sparsity.append( (sparsity_list12_avg[i].item()) / 1 )
    # ***************** sparsity calculation ***************** #

    return test_acc, average_sparsity, selectivity_avg_list, selectivity_std_list

In [11]:
# Seed 100 

torch.manual_seed(100)
np.random.seed(100)

In [12]:
# 512 neurons 
model_factory('Adadelta', 100, 512)
model_factory('Adagrad', 100, 512)
model_factory('SGD', 100, 512)
model_factory('Adam', 100, 512)

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=512, bias=True)
  (linear_2): Linear(in_features=512, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: Adadelta (
Parameter Group 0
    eps: 1e-06
    lr: 1.0
    rho: 0.9
    weight_decay: 0
)

Epoch: 1/30, Train Loss: 19492.49588356, Test Loss: 38.55574384, Test Accuracy: 0.95150000

Epoch: 2/30, Train Loss: 10630.15119109, Test Loss: 28.23871937, Test Accuracy: 0.96390000

Epoch: 3/30, Train Loss: 7918.65379271, Test Loss: 24.30461866, Test Accuracy: 0.96970000

Epoch: 4/30, Train Loss: 6572.17004709, Test Loss: 21.59482770, Test Accuracy: 0.97360000

Epoch: 5/30, Train Loss: 5553.43920123, Test Loss: 24.16887339, Test Accuracy: 0.97050000

Epoch: 6/30, Train Loss: 4872.28187749, Test Loss: 20.03023274, Test Accuracy: 0.97650000

Epoch: 7/30, Train Loss: 4357.85218723, Test Loss: 18.17426659, Test Accuracy: 0.97780000

Epoch: 8/30, Train Loss: 3858.19409329, Test Loss: 18.73996834, Test Accuracy:

In [13]:
# 640 neurons 
model_factory('Adadelta', 100, 640)
model_factory('Adagrad', 100, 640)
model_factory('SGD', 100, 640)
model_factory('Adam', 100, 640)

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=640, bias=True)
  (linear_2): Linear(in_features=640, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: Adadelta (
Parameter Group 0
    eps: 1e-06
    lr: 1.0
    rho: 0.9
    weight_decay: 0
)

Epoch: 1/30, Train Loss: 19772.07822771, Test Loss: 40.85411794, Test Accuracy: 0.94690000

Epoch: 2/30, Train Loss: 10633.45523165, Test Loss: 27.28101790, Test Accuracy: 0.96640000

Epoch: 3/30, Train Loss: 7825.16866588, Test Loss: 23.69824867, Test Accuracy: 0.97170000

Epoch: 4/30, Train Loss: 6327.01250547, Test Loss: 22.81873020, Test Accuracy: 0.97320000

Epoch: 5/30, Train Loss: 5376.14017060, Test Loss: 20.31283486, Test Accuracy: 0.97520000

Epoch: 6/30, Train Loss: 4548.55535731, Test Loss: 19.04140437, Test Accuracy: 0.97740000

Epoch: 7/30, Train Loss: 4068.27385640, Test Loss: 18.38303256, Test Accuracy: 0.97860000

Epoch: 8/30, Train Loss: 3568.83286647, Test Loss: 18.32878473, Test Accuracy:

In [12]:
# 768 neurons 
model_factory('Adadelta', 100, 768)

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=768, bias=True)
  (linear_2): Linear(in_features=768, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: Adadelta (
Parameter Group 0
    eps: 1e-06
    lr: 1.0
    rho: 0.9
    weight_decay: 0
)

Epoch: 1/30, Train Loss: 19490.64012896, Test Loss: 36.36237110, Test Accuracy: 0.95690000

Epoch: 2/30, Train Loss: 10037.70986050, Test Loss: 27.27119897, Test Accuracy: 0.96850000

Epoch: 3/30, Train Loss: 7774.59226534, Test Loss: 22.59575054, Test Accuracy: 0.97310000

Epoch: 4/30, Train Loss: 6486.29624911, Test Loss: 26.57668791, Test Accuracy: 0.96970000

Epoch: 5/30, Train Loss: 5342.57286470, Test Loss: 20.34404875, Test Accuracy: 0.97460000

Epoch: 6/30, Train Loss: 4791.99138777, Test Loss: 23.03247804, Test Accuracy: 0.97370000

Epoch: 7/30, Train Loss: 4326.52349769, Test Loss: 22.52394989, Test Accuracy: 0.97500000

Epoch: 8/30, Train Loss: 3706.39730091, Test Loss: 19.27344558, Test Accuracy:

In [13]:
model_factory('Adagrad', 100, 768)

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=768, bias=True)
  (linear_2): Linear(in_features=768, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: Adagrad (
Parameter Group 0
    eps: 1e-10
    initial_accumulator_value: 0
    lr: 0.1
    lr_decay: 0
    weight_decay: 0
)

Epoch: 1/30, Train Loss: 12310.47955465, Test Loss: 23.23767545, Test Accuracy: 0.96540000

Epoch: 2/30, Train Loss: 5296.20115186, Test Loss: 18.98711549, Test Accuracy: 0.97080000

Epoch: 3/30, Train Loss: 3618.89521517, Test Loss: 16.48115788, Test Accuracy: 0.97570000

Epoch: 4/30, Train Loss: 2601.77808075, Test Loss: 15.11687978, Test Accuracy: 0.97680000

Epoch: 5/30, Train Loss: 1941.14794338, Test Loss: 14.66719693, Test Accuracy: 0.97830000

Epoch: 6/30, Train Loss: 1475.87267528, Test Loss: 14.04351352, Test Accuracy: 0.97840000

Epoch: 7/30, Train Loss: 1113.34357712, Test Loss: 13.35084914, Test Accuracy: 0.98060000

Epoch: 8/30, Train Loss: 870.16921227, Test 

In [14]:
model_factory('SGD', 100, 768)

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=768, bias=True)
  (linear_2): Linear(in_features=768, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: SGD (
Parameter Group 0
    dampening: 0
    lr: 0.1
    momentum: 0
    nesterov: False
    weight_decay: 0
)

Epoch: 1/30, Train Loss: 15046.75034823, Test Loss: 23.75220769, Test Accuracy: 0.96290000

Epoch: 2/30, Train Loss: 6374.46905226, Test Loss: 20.74123894, Test Accuracy: 0.96980000

Epoch: 3/30, Train Loss: 4377.64712033, Test Loss: 17.86008700, Test Accuracy: 0.97310000

Epoch: 4/30, Train Loss: 3361.76002316, Test Loss: 19.48922984, Test Accuracy: 0.97100000

Epoch: 5/30, Train Loss: 2525.48166197, Test Loss: 16.72660741, Test Accuracy: 0.97620000

Epoch: 6/30, Train Loss: 1988.05364860, Test Loss: 20.91065948, Test Accuracy: 0.97240000

Epoch: 7/30, Train Loss: 1564.14087760, Test Loss: 18.09141067, Test Accuracy: 0.97630000

Epoch: 8/30, Train Loss: 1179.33621938, Test Loss: 16.37916

In [15]:
model_factory('Adam', 100, 768)

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=768, bias=True)
  (linear_2): Linear(in_features=768, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

Epoch: 1/30, Train Loss: 11697.53532559, Test Loss: 21.62471983, Test Accuracy: 0.96780000

Epoch: 2/30, Train Loss: 5148.63165765, Test Loss: 21.63483844, Test Accuracy: 0.96920000

Epoch: 3/30, Train Loss: 3649.98695950, Test Loss: 17.56349295, Test Accuracy: 0.98010000

Epoch: 4/30, Train Loss: 2753.91359644, Test Loss: 18.32065837, Test Accuracy: 0.98060000

Epoch: 5/30, Train Loss: 2176.66693839, Test Loss: 22.47109790, Test Accuracy: 0.97900000

Epoch: 6/30, Train Loss: 1657.33642741, Test Loss: 21.58839745, Test Accuracy: 0.98110000

Epoch: 7/30, Train Loss: 1557.31070642, Test Loss: 22.34511244, Test Accuracy: 0.98040000

Epoch: 8/30, Train Loss: 1234.89659642, Test Loss: 